<a href="https://colab.research.google.com/github/venkatesh-manikumar/Datascience-Assignments/blob/main/Predict_The_Flight_Ticket_Price_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
from google.colab import drive
##drive.mount('/content/drive')

In [106]:
import pandas as pd
import holidays
from sklearn.linear_model import LinearRegression

In [91]:
train_data = pd.read_excel('/content/drive/MyDrive/DataScience/Flight_price_prediction/Data_Train.xlsx')
train_data.shape

(10683, 11)

In [94]:
train_data['Journey_date'] = pd.to_datetime(train_data['Date_of_Journey'],dayfirst=True)
# df.assign(Percentage = lambda x: (x['Total_Marks'] /500 * 100))
# df = df.apply(lambda x: np.square(x) if x.name == 'd' else x, axis=1)
# train_data['dayofweek'] =train_data['Journey_date'].apply(lambda x:x.dayofweek)
train_data['Day'] =train_data['Journey_date'].apply(lambda x:x.day_name())
train_data['Day_of_week'] =train_data['Journey_date'].apply(lambda x:x.isoweekday())
train_data

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_date,Dayname,Weekday,isHoliday,Day,Day_of_week
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,2019-03-24,Sunday,7,0,Sunday,7
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,2019-05-01,Wednesday,3,1,Wednesday,3
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,2019-06-09,Sunday,7,1,Sunday,7
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,2019-05-12,Sunday,7,0,Sunday,7
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,2019-03-01,Friday,5,0,Friday,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107,2019-04-09,Tuesday,2,0,Tuesday,2
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145,2019-04-27,Saturday,6,0,Saturday,6
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229,2019-04-27,Saturday,6,0,Saturday,6
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648,2019-03-01,Friday,5,0,Friday,5


In [93]:
India_holidays = holidays.India()
#print('25-12-2024' in India_holidays)
train_data['isHoliday'] =train_data['Journey_date'].apply(lambda x: 1 if x in India_holidays else 0)
train_data.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_date,Dayname,Weekday,isHoliday
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,2019-03-24,Sunday,7,0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,2019-05-01,Wednesday,3,1
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,2019-06-09,Sunday,7,1
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,2019-05-12,Sunday,7,0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,2019-03-01,Friday,5,0
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873,2019-06-24,Monday,1,0
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087,2019-03-12,Tuesday,2,0
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270,2019-03-01,Friday,5,0
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087,2019-03-12,Tuesday,2,0
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625,2019-05-27,Monday,1,0


In [104]:
for date,event in India_holidays.items():
  print(date,event)

2019-01-14 Makar Sankranti / Pongal
2019-01-26 Republic Day
2019-08-15 Independence Day
2019-10-02 Gandhi Jayanti
2019-05-01 Labour Day
2019-10-27 Diwali
2019-03-21 Holi
2019-09-09 Day of Ashura* (*estimated)
2019-11-09 Mawlid* (*estimated)
2019-06-04 Eid ul-Fitr* (*estimated)
2019-06-05 Eid ul-Fitr* (*estimated)
2019-08-11 Eid al-Adha* (*estimated)
2019-08-12 Eid al-Adha* (*estimated)
2019-04-14 Palm Sunday
2019-04-19 Good Friday
2019-04-21 Easter Sunday
2019-06-09 Feast of Pentecost
2019-12-25 Christmas Day


In [119]:
lin_reg = LinearRegression()
train_labels = train_data['Price'].copy()
train= train_data.drop('Price',axis=1)
lin_reg.fit(train,train_labels)

ValueError: ignored